In [ ]:
gcs_transcripts_directory="gs://mickael-dm-stt/transcripts" #GCS URI where transcripts are stored in format gs://bucket_name/path
gcs_ground_truth_directory="gs://mickael-dm-stt/ground_truth" #GCS URI where ground truth transcripts are stored in format gs://bucket_name/path

In [ ]:
import re

model_folders=!(gsutil ls {gcs_transcripts_directory})
models=[]

for folder in model_folders:
    model=re.search("([^/]+)/$",folder).group(1)
    models.append(model)

print(models)

In [ ]:
import re
from google.cloud import storage

storage_client = storage.Client()
ground_truth_bucket_name,ground_truth_path=re.search(f"gs://([^/]+)/(.*)",gcs_ground_truth_directory).group(1,2)
ground_truth_bucket = storage_client.bucket(ground_truth_bucket_name)
transcripts_bucket_name,transcripts_path=re.search(f"gs://([^/]+)/(.*)",gcs_transcripts_directory).group(1,2)
transcripts_bucket = storage_client.bucket(transcripts_bucket_name)
videos=[]

ground_truth_uris=!(gsutil ls {gcs_ground_truth_directory})
for uri in ground_truth_uris:
    
    video_id=re.search("([^/]+).txt$",uri).group(1)
    print(f"Video ID: {video_id}")
    
    
    ground_truth_blob=ground_truth_bucket.blob(f"{ground_truth_path}/{video_id}.txt")
    ground_truth=ground_truth_blob.download_as_text()
    print(f"Ground truth: {ground_truth}")
    
    video={
        "id": video_id,
        "ground_truth": ground_truth,
        "transcripts": []
    }
    
    for model in models:
        
        transcript_blob=transcripts_bucket.blob(f"{transcripts_path}/{model}/txt/{video_id}.txt")
        transcript=transcript_blob.download_as_text()
        print(f"Transcript for {model}: {transcript}")
        video["transcripts"].append([model, transcript])
    videos.append(video)

In [ ]:
import os
from simple_wer_v2 import SimpleWER
import pandas as pd

video_ids=[]
wer_scores=[]

for video in videos:
    print(f"Video: {video['id']}")
    video_ids.append(video["id"])
    wer_score={}
    for transcript in video["transcripts"]:
        
        werClient=SimpleWER()
        werClient.AddHypRef(transcript[1], video["ground_truth"])
        wer=werClient.GetWER()
        wer_score[transcript[0]]=wer
        print(f"WER score for {transcript[0]}: {wer}")
        html="<body><html><div>"+werClient.aligned_htmls[0]+"</div></body></html>"
        
        if not os.path.exists(f"wer_diagnosis/{transcript[0]}/"):
            os.makedirs(f"wer_diagnosis/{transcript[0]}/")
        file=open(f"wer_diagnosis/{transcript[0]}/{video['id']}.html","w")
        #file=open("wer_diagnosis/"+transcript[0]+"/"+video['id']+".html","w")
        file.write(html)
        file.close()
    wer_scores.append(wer_score)
print(wer_scores)

df=pd.DataFrame(wer_scores, index=video_ids)
df